In [35]:
import pandas as pd
from datetime import timedelta
import numpy as np

In [53]:
drug = pd.read_csv(r"C:\Users\cmuh1\VSCode\jupter\warfarin_Source\warfarin_Drug.csv")
database = pd.read_excel(r'C:\Users\cmuh1\VSCode\warfarin總表-0311(第七版).xlsx')

In [54]:
drug['OrderTime1'] = pd.to_datetime(drug['OrderTime']).dt.date
drug['EndTime1'] = pd.to_datetime(drug['EndTime']).dt.date
drug['OrderTime_min'] = drug.groupby('PatientNo')['OrderTime1'].transform(min)

In [55]:
drug['MedCode'] = drug['MedCode'].str.strip()
conditions=[(drug.MedCode == 'TCOUMAD'),
 (drug.MedCode == 'TWARFA1'),
 (drug.MedCode == 'TWARFA11'),
 (drug.MedCode == 'TWARFA12'),
 (drug.MedCode == 'TWARFAR'),
 (drug.MedCode == 'TWARFAR1'),
 (drug.MedCode == 'TWARFAR2'),
 ]
values = [ 5 , 1 , 1, 1, 5, 5, 5]

drug['Dose(mg/tab)'] = np.select(conditions,values)
drug['Dose(mg/tab)'] = drug['Dose(mg/tab)'].multiply(drug['Dosage'])

In [57]:
drug['timediff'] = drug['EndTime1'] - drug['OrderTime_min']
drug['drug_day'] = drug['EndTime1'] - drug['OrderTime1']
# drug['drug_day1'] = np.where(drug.UsageNo == 9000 , pd.Timedelta(days=1) , drug.drug_day.dt.days )
drug = drug[drug['timediff'] <timedelta(days=180)]


In [59]:
OrderNo = pd.DataFrame(database.OrderNo)
OrderNo['OrderNo1'] = OrderNo.OrderNo

In [60]:
drug = pd.merge(drug , OrderNo,how='left' , on='OrderNo')

In [ ]:
drug['OrderTime_min5'] = drug.OrderTime_min +timedelta(days=5)
drug['OrderTime_min7'] = drug.OrderTime_min +timedelta(days=7)
drug['OrderTime_min14'] = drug.OrderTime_min +timedelta(days=14)
drug['OrderTime_min30'] = drug.OrderTime_min +timedelta(days=30)
drug['OrderTime_min60'] = drug.OrderTime_min +timedelta(days=60)
drug['OrderTime_min90'] = drug.OrderTime_min +timedelta(days=90)
drug

In [68]:
drug5 = drug[drug.OrderTime1 <= drug.OrderTime_min5]
drug5 = drug5.drop(columns=['OrderTime_min7','OrderTime_min14','OrderTime_min30','OrderTime_min60','OrderTime_min90'])
drug7 = drug[drug.OrderTime1 <= drug.OrderTime_min7]
drug7 = drug7.drop(columns=['OrderTime_min5','OrderTime_min14','OrderTime_min30','OrderTime_min60','OrderTime_min90'])
drug14 = drug[drug.OrderTime1 <= drug.OrderTime_min14]
drug14 = drug14.drop(columns=['OrderTime_min5','OrderTime_min7','OrderTime_min30','OrderTime_min60','OrderTime_min90'])
drug30 = drug[drug.OrderTime1 <= drug.OrderTime_min30]
drug30 = drug30.drop(columns=['OrderTime_min5','OrderTime_min7','OrderTime_min14','OrderTime_min60','OrderTime_min90'])
drug60 = drug[drug.OrderTime1 <= drug.OrderTime_min60]
drug60 = drug60.drop(columns=['OrderTime_min5','OrderTime_min7','OrderTime_min14','OrderTime_min30','OrderTime_min90'])
drug90 = drug[drug.OrderTime1 <= drug.OrderTime_min90]
drug90 = drug90.drop(columns=['OrderTime_min5','OrderTime_min7','OrderTime_min14','OrderTime_min30','OrderTime_min60'])

In [70]:
drug5 = drug5.sort_values(['PatientNo','OrderTime'])
drug5['OrderTime'] = pd.to_datetime(drug5['OrderTime'])
drug5['OrderTime_diff'] = drug5.groupby('PatientNo')['OrderTime1'].diff().fillna(pd.Timedelta(0))
drug5['sameday'] = drug5.groupby(['PatientNo', 'OrderTime1','OrderTime_diff'])['OrderTime'].transform('max')
drug5 = drug5[drug5['OrderTime'] == drug5.sameday]
drug5 = drug5.drop_duplicates()

drug5['drug_day'] = np.where(drug5.UsageNo == 9000 , 1 , drug5.drug_day.dt.days )
drug5['Sum_Dose'] = np.where(drug5['EndTime1']<=drug5['OrderTime_min5'], drug5['Dose(mg/tab)']*drug5['drug_day'],
                              (pd.to_numeric((drug5['OrderTime_min5']-drug5['OrderTime1']).dt.days, downcast='integer'))*drug5['Dose(mg/tab)'])

# 5天平均劑量
Dose_mean5 = drug5.groupby('PatientNo')['Sum_Dose'].sum()/5

In [71]:
drug7 = drug7.sort_values(['PatientNo','OrderTime'])
drug7['OrderTime'] = pd.to_datetime(drug7['OrderTime'])
drug7['OrderTime_diff'] = drug7.groupby('PatientNo')['OrderTime1'].diff().fillna(pd.Timedelta(0))
drug7['sameday'] = drug7.groupby(['PatientNo', 'OrderTime1','OrderTime_diff'])['OrderTime'].transform('max')
drug7 = drug7[drug7['OrderTime'] == drug7.sameday]
drug7 = drug7.drop_duplicates()

drug7['drug_day'] = np.where(drug7.UsageNo == 9000 , 1 , drug7.drug_day.dt.days )
drug7['Sum_Dose'] = np.where(drug7['EndTime1']<=drug7['OrderTime_min7'], drug7['Dose(mg/tab)']*drug7['drug_day'],
                              (pd.to_numeric((drug7['OrderTime_min7']-drug7['OrderTime1']).dt.days, downcast='integer'))*drug7['Dose(mg/tab)'])

# 7天平均劑量
Dose_mean7 = drug7.groupby('PatientNo')['Sum_Dose'].sum()/7

In [72]:
drug14 = drug14.sort_values(['PatientNo','OrderTime'])
drug14['OrderTime'] = pd.to_datetime(drug14['OrderTime'])
drug14['OrderTime_diff'] = drug14.groupby('PatientNo')['OrderTime1'].diff().fillna(pd.Timedelta(0))
drug14['sameday'] = drug14.groupby(['PatientNo', 'OrderTime1','OrderTime_diff'])['OrderTime'].transform('max')
drug14 = drug14[drug14['OrderTime'] == drug14.sameday]
drug14 = drug14.drop_duplicates()

drug14['drug_day'] = np.where(drug14.UsageNo == 9000 , 1 , drug14.drug_day.dt.days )
drug14['Sum_Dose'] = np.where(drug14['EndTime1']<=drug14['OrderTime_min14'], drug14['Dose(mg/tab)']*drug14['drug_day'],
                              (pd.to_numeric((drug14['OrderTime_min14']-drug14['OrderTime1']).dt.days, downcast='integer'))*drug14['Dose(mg/tab)'])

# 14天平均劑量
Dose_mean14 = drug14.groupby('PatientNo')['Sum_Dose'].sum()/14

In [73]:
drug30 = drug30.sort_values(['PatientNo','OrderTime'])
drug30['OrderTime'] = pd.to_datetime(drug30['OrderTime'])
drug30['OrderTime_diff'] = drug30.groupby('PatientNo')['OrderTime1'].diff().fillna(pd.Timedelta(0))
drug30['sameday'] = drug30.groupby(['PatientNo', 'OrderTime1','OrderTime_diff'])['OrderTime'].transform('max')
drug30 = drug30[drug30['OrderTime'] == drug30.sameday]
drug30 = drug30.drop_duplicates()

drug30['drug_day'] = np.where(drug30.UsageNo == 9000 , 1 , drug30.drug_day.dt.days )
drug30['Sum_Dose'] = np.where(drug30['EndTime1']<=drug30['OrderTime_min30'], drug30['Dose(mg/tab)']*drug30['drug_day'],
                              (pd.to_numeric((drug30['OrderTime_min30']-drug30['OrderTime1']).dt.days, downcast='integer'))*drug30['Dose(mg/tab)'])

# 30天平均劑量
Dose_mean30 = drug30.groupby('PatientNo')['Sum_Dose'].sum()/30



In [74]:
drug60 = drug60.sort_values(['PatientNo','OrderTime'])
drug60['OrderTime'] = pd.to_datetime(drug60['OrderTime'])
drug60['OrderTime_diff'] = drug60.groupby('PatientNo')['OrderTime1'].diff().fillna(pd.Timedelta(0))
drug60['sameday'] = drug60.groupby(['PatientNo', 'OrderTime1','OrderTime_diff'])['OrderTime'].transform('max')
drug60 = drug60[drug60['OrderTime'] == drug60.sameday]
drug60 = drug60.drop_duplicates()

drug60['drug_day'] = np.where(drug60.UsageNo == 9000 , 1 , drug60.drug_day.dt.days )
drug60['Sum_Dose'] = np.where(drug60['EndTime1']<=drug60['OrderTime_min60'], drug60['Dose(mg/tab)']*drug60['drug_day'],
                              (pd.to_numeric((drug60['OrderTime_min60']-drug60['OrderTime1']).dt.days, downcast='integer'))*drug60['Dose(mg/tab)'])

# 60天平均劑量
Dose_mean60 = drug60.groupby('PatientNo')['Sum_Dose'].sum()/60

In [75]:
drug90 = drug90.sort_values(['PatientNo','OrderTime'])
drug90['OrderTime'] = pd.to_datetime(drug90['OrderTime'])
drug90['OrderTime_diff'] = drug90.groupby('PatientNo')['OrderTime1'].diff().fillna(pd.Timedelta(0))
drug90['sameday'] = drug90.groupby(['PatientNo', 'OrderTime1','OrderTime_diff'])['OrderTime'].transform('max')
drug90 = drug90[drug90['OrderTime'] == drug90.sameday]
drug90 = drug90.drop_duplicates()

drug90['drug_day'] = np.where(drug90.UsageNo == 9000 , 1 , drug90.drug_day.dt.days )
drug90['Sum_Dose'] = np.where(drug90['EndTime1']<=drug90['OrderTime_min90'], drug90['Dose(mg/tab)']*drug90['drug_day'],
                              (pd.to_numeric((drug90['OrderTime_min90']-drug90['OrderTime1']).dt.days, downcast='integer'))*drug90['Dose(mg/tab)'])

# 90天平均劑量
Dose_mean90 = drug90.groupby('PatientNo')['Sum_Dose'].sum()/90

In [77]:
Dose_mean5

PatientNo
100000000498    2.5
100000001159    1.6
100000001395    4.0
100000001714    2.5
100000002675    2.5
               ... 
100035861165    1.4
100035863490    1.8
100035892222    2.0
100035901923    0.5
100036202057    5.0
Name: Sum_Dose, Length: 2613, dtype: float64

In [78]:
Dose_mean7

PatientNo
100000000498    2.500000
100000001159    2.571429
100000001395    4.285714
100000001714    2.500000
100000002675    2.500000
                  ...   
100035861165    1.642857
100035863490    1.785714
100035892222    2.000000
100035901923    0.357143
100036202057    5.000000
Name: Sum_Dose, Length: 2613, dtype: float64

In [79]:
Dose_mean14

PatientNo
100000000498    2.500000
100000001159    4.428571
100000001395    4.107143
100000001714    2.142857
100000002675    2.500000
                  ...   
100035861165    2.785714
100035863490    1.892857
100035892222    1.571429
100035901923    0.178571
100036202057    2.500000
Name: Sum_Dose, Length: 2613, dtype: float64

In [80]:
Dose_mean30

PatientNo
100000000498    2.500000
100000001159    6.333333
100000001395    3.250000
100000001714    1.000000
100000002675    1.333333
                  ...   
100035861165    4.466667
100035863490    3.150000
100035892222    0.733333
100035901923    0.083333
100036202057    1.166667
Name: Sum_Dose, Length: 2613, dtype: float64

In [81]:
Dose_mean60

PatientNo
100000000498    2.500000
100000001159    5.616667
100000001395    2.875000
100000001714    0.500000
100000002675    1.916667
                  ...   
100035861165    2.233333
100035863490    1.575000
100035892222    0.366667
100035901923    0.041667
100036202057    0.583333
Name: Sum_Dose, Length: 2613, dtype: float64

In [82]:
Dose_mean90

PatientNo
100000000498    2.500000
100000001159    3.744444
100000001395    2.750000
100000001714    0.333333
100000002675    2.111111
                  ...   
100035861165    1.488889
100035863490    1.050000
100035892222    0.244444
100035901923    0.027778
100036202057    0.388889
Name: Sum_Dose, Length: 2613, dtype: float64

In [90]:
Dose_all = pd.merge(Dose_mean5 , Dose_mean7 , how='left' ,on='PatientNo')
Dose_all = Dose_all.rename(columns={'Sum_Dose_x':'Dose_mean5' ,'Sum_Dose_y':'Dose_mean7' })
Dose_all = pd.merge(Dose_all , Dose_mean14 , how='left' ,on='PatientNo')
Dose_all = Dose_all.rename(columns={'Sum_Dose':'Dose_mean14' })
Dose_all = pd.merge(Dose_all , Dose_mean30 , how='left' ,on='PatientNo')
Dose_all = Dose_all.rename(columns={'Sum_Dose':'Dose_mean30' })
Dose_all = pd.merge(Dose_all , Dose_mean60 , how='left' ,on='PatientNo')
Dose_all = Dose_all.rename(columns={'Sum_Dose':'Dose_mean60' })
Dose_all = pd.merge(Dose_all , Dose_mean90 , how='left' ,on='PatientNo')
Dose_all = Dose_all.rename(columns={'Sum_Dose':'Dose_mean90' })
Dose_all.to_excel(r'C:\Users\cmuh1\VSCode\Dose_mean.xlsx')

In [89]:
dose_mean = [Dose_mean5, Dose_mean7, Dose_mean14, Dose_mean30, Dose_mean60, Dose_mean90]

Dose = pd.concat(dose_mean, join='inner', axis=1)
Dose

,Sum_Dose,Sum_Dose,Sum_Dose,Sum_Dose,Sum_Dose,Sum_Dose
PatientNo,,,,,,
100000000498,2.5,2.500000,2.500000,2.500000,2.500000,2.500000
100000001159,1.6,2.571429,4.428571,6.333333,5.616667,3.744444
100000001395,4.0,4.285714,4.107143,3.250000,2.875000,2.750000
100000001714,2.5,2.500000,2.142857,1.000000,0.500000,0.333333
100000002675,2.5,2.500000,2.500000,1.333333,1.916667,2.111111
...,...,...,...,...,...,...
100035861165,1.4,1.642857,2.785714,4.466667,2.233333,1.488889
100035863490,1.8,1.785714,1.892857,3.150000,1.575000,1.050000
100035892222,2.0,2.000000,1.571429,0.733333,0.366667,0.244444
